<a href="https://colab.research.google.com/github/herysedra/covid19-mankaiza-clone/blob/andrana/scripts/paper/simple_blocks/simple_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pymc3 as pm
import theano.tensor as tt
import theano
import datetime
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib
import pickle


In [0]:
num_days = 5

a = tt.ones(num_days-1)
b = np.array(a)
b

array(Alloc.0, dtype=object)

In [0]:
print(tt.ones(num_days-1))

In [0]:
type(tt.ones(num_days-1))

theano.tensor.var.TensorVariable

In [0]:
tt.ones(num_days-1).shape

Shape.0

In [0]:
2*tt.ones(num_days-1)

Elemwise{mul,no_inplace}.0

In [0]:
print(2*tt.ones(num_days-1))

Elemwise{mul,no_inplace}.0


In [0]:
type(2*tt.ones(num_days-1))

theano.tensor.var.TensorVariable

In [0]:
2*tt.ones(num_days-1).shape

Elemwise{mul,no_inplace}.0

In [0]:
a = np.array([1, 2, 3, 4])
a

array([1, 2, 3, 4])

In [0]:
b = range(10)
type(a)

numpy.ndarray

In [0]:
A = np.array([[1,2,3,4],[5,6,7,8]])
print(A)
A[:,2] # returns the third columm

[[1 2 3 4]
 [5 6 7 8]]


array([3, 7])

In [4]:
# Example 1 for theano.scan (for the construction of the SIR_model)

k = tt.iscalar("k")
A = tt.vector("A")

# Symbolic description of the result
result, updates = theano.scan(fn=lambda prior_result, A: prior_result * A,
                              outputs_info=tt.ones_like(A),
                              non_sequences=A,
                              n_steps=k)

# We only care about A**k, but scan has provided us with A**1 through A**k.
# result.shape is equal to (A.shape)**k 
# Discard the values that we don't care about. Scan is smart enough to
# notice this and not waste memory saving them.

final_result = result[-1]

#[-1] retrieves the last iteration result from the output 'result'

# compiled function that returns A**k
power = theano.function(inputs=[A,k], outputs=final_result, updates=updates)
bb = theano.function([A,k], outputs=result)

d = tt.scalar('d')

print(power(range(10),2))
print(power(range(10),4))

print(bb(range(10),4))
# notice the absence of [-1] in the outputs of bb
# print(bb(d,2)) will give error: no symbolic output

[ 0.  1.  4.  9. 16. 25. 36. 49. 64. 81.]
[0.000e+00 1.000e+00 1.600e+01 8.100e+01 2.560e+02 6.250e+02 1.296e+03
 2.401e+03 4.096e+03 6.561e+03]
[[0.000e+00 1.000e+00 2.000e+00 3.000e+00 4.000e+00 5.000e+00 6.000e+00
  7.000e+00 8.000e+00 9.000e+00]
 [0.000e+00 1.000e+00 4.000e+00 9.000e+00 1.600e+01 2.500e+01 3.600e+01
  4.900e+01 6.400e+01 8.100e+01]
 [0.000e+00 1.000e+00 8.000e+00 2.700e+01 6.400e+01 1.250e+02 2.160e+02
  3.430e+02 5.120e+02 7.290e+02]
 [0.000e+00 1.000e+00 1.600e+01 8.100e+01 2.560e+02 6.250e+02 1.296e+03
  2.401e+03 4.096e+03 6.561e+03]]


In [0]:
# Example 2 for theano.scan
x = tt.vector('x')

def square(x):
  return x*x

outputs, updates = theano.scan(
  fn=square,
  sequences=x,
  n_steps=x.shape[0],
)

square_op = theano.function(
  inputs=[x],
  outputs=[outputs],
)

o_val = square_op(np.array([1, 2, 3, 4, 5]))

print("output:", o_val)

# theano.scan has symbolic outputs which can be taken as outputs of another function
# and the sequence_variable of the scan as its inputs 

output: [array([ 1.,  4.,  9., 16., 25.])]


In [0]:
# Example 3 for theano.scan
constant = tt.dscalar('constant')
n_iters = tt.iscalar('n_iters')
start = tt.dscalar('start')

results, _ = theano.scan(lambda result, constant: result + constant,
                         outputs_info=[start], non_sequences=[constant], n_steps=n_iters)

f = theano.function([start, constant, n_iters], results[-1])
print('f(0,2,5): {0}'.format(f(0, 2, 5)))

f(0,2,5): 10.0


In [0]:
# concatenate function (for the construction of new_cases_inferred)

u = tt.vector('u')
#u = [1, 2, 3, 4]
v = tt.vector('v')
#v = [11, 12, 13, 14]

w = tt.fmatrix('w')
w = tt.concatenate([u[-4:], v])
# notice the [-4:]
f = theano.function([u,v], w)
f([1, 2, 3, 4],[11, 12, 13, 14])

array([ 1.,  2.,  3.,  4., 11., 12., 13., 14.])

In [0]:
u = tt.vector('u')
#u = [1, 2, 3, 4]
v = tt.vector('v')
#v = [11, 12, 13, 14]

w = tt.fmatrix('w')
w = tt.concatenate([u[-2:], v])
# notive the [-3:] and compare its effect with the one of [-4:]
f = theano.function([u,v], w)
f([1, 2, 3, 4],[11, 12, 13, 14])

array([ 3.,  4., 11., 12., 13., 14.])

In [0]:
u = tt.vector('u')
#u = [1, 2, 3, 4]
v = tt.vector('v')
#v = [11, 12, 13, 14]

w = tt.fmatrix('w')
w = tt.concatenate([u, v])
f = theano.function([u,v], w)
f([1, 2, 3, 4],[11, 12, 13, 14])

array([ 1.,  2.,  3.,  4., 11., 12., 13., 14.])

In [0]:
np.ones(5)

array([1., 1., 1., 1., 1.])

In [0]:
s = (2,2)
np.ones(s)

array([[1., 1.],
       [1., 1.]])

In [0]:
#Ex for tt.maximum
x = tt.vector('x')
y = tt.vector('y')
z = tt.maximum(x,y)
f = theano.function([x,y], z)
print(f((1,1),(2,1)))
print(f((1,3),(2,1)))
print(f((1,3),(2,4)))
# Returns a variable representing the maximum element by element of a and b

[2. 1.]
[2. 3.]
[2. 4.]


In [0]:
x = tt.vector('x')
y = tt.vector('y')
z = tt.ones(3)
z[2]

Subtensor{int64}.0

In [0]:
# Ex for tt.abs_
x = tt.matrix('x')
y = tt.matrix('y')
z = tt.abs_(x - 1)
f = theano.function([x], z)
A = np.array([[1,2,3,4],[5,6,7,-8]])
f(A)
# returns the absolute value, not the norm
# notice that, in 'x - 1', '1' is the matrix filled with ones

array([[0., 1., 2., 3.],
       [4., 5., 6., 9.]])

In [0]:
def make_delay_matrix(n_rows, n_columns, initial_delay=0):
    """
    Has in each entry the delay between the input with size n_rows and the output
    with size n_columns
    """
    size = max(n_rows, n_columns)
    mat = np.zeros((size, size))
    for i in range(size):
        diagonal = np.ones(size-i)*(initial_delay + i)
        mat += np.diag(diagonal, i)
    for i in range(1, size):
        diagonal = np.ones(size-i)*(initial_delay - i)
        mat += np.diag(diagonal, -i)
    return mat[:n_rows, :n_columns]
nr = 10
nc = 15
    ff = make_delay_matrix(n_rows, n_columns, initial_delay)

